# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [182]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [183]:
# loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true')

# loans
import requests
import gzip
import pandas as pd
from io import BytesIO
import csv

url = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'

# Obtener el contenido del archivo (ignorando la verificación SSL)
response = requests.get(url, verify=False)

# Usar BytesIO para tratar el contenido binario
compressed_file = BytesIO(response.content)

# Inicializar una lista para almacenar las filas problemáticas
bad_lines = []

# Descomprimir y leer el CSV
with gzip.open(compressed_file, 'rt', encoding='utf-8') as f:
    # Detectar automáticamente el delimitador
    sample = f.read(2048)
    f.seek(0)
    dialect = csv.Sniffer().sniff(sample)

    # Función personalizada para manejar las filas problemáticas
    def handle_bad_lines(row):
        bad_lines.append(row)
        return None  # Excluir la fila problemática del DataFrame principal

    # Cargar el CSV usando el motor 'python' y capturar filas problemáticas
    loans = pd.read_csv(
        f,
        delimiter=dialect.delimiter,
        on_bad_lines=handle_bad_lines,
        engine='python'  # Usar el motor 'python' para permitir la función personalizada
    )



# Mostrar las filas problemáticas, si existen
if bad_lines:
    print("\nProblematic Rows:")
    for line in bad_lines:
        print(line)
else:
    print("\nNo problematic rows found.")

# Mostrar la forma (dimensiones) del DataFrame
print(f"\nDataFrame Shape: {loans.shape}")

loans
print(loans.shape)



/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advi


No problematic rows found.

DataFrame Shape: (100000, 151)
(100000, 151)


In [184]:
df = pd.DataFrame(loans)
df

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:


# Get the list of columns
columns = df.columns
print(columns) 
print(loans.columns.tolist())


Index(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'fico_range_low', 'fico_range_high',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'last_fico_range_high', 'last_fico_range_low',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'po

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [186]:
# column_types =
unique_dtypes = df.dtypes.unique()

print("Unique Data Types in the DataFrame:")
for dtype in unique_dtypes:
    print(dtype)

Unique Data Types in the DataFrame:
int64
float64
object


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [187]:
! pip install openpyxl
! pip install certifi
! pip install --upgrade certifi

In [188]:


##datos_dict = pd.read_excel(
   ## 'https://resources.lendingclub.com/LCDataDictionary.xlsx')
##datos_dict.columns = ['feature', 'description']

import certifi

# Download the Excel file with SSL verification
url = 'https://resources.lendingclub.com/LCDataDictionary.xlsx'
response = requests.get(url, verify=certifi.where())

# Save the content to a local file
with open('LCDataDictionary.xlsx', 'wb') as f:
    f.write(response.content)

# Load the Excel file with the openpyxl engine
datos_dict = pd.read_excel('LCDataDictionary.xlsx', engine='openpyxl')

# Rename columns
datos_dict.columns = ['feature', 'description']

# Display the DataFrame
print(datos_dict.head())

                feature                                        description
0        acc_now_delinq  The number of accounts on which the borrower i...
1  acc_open_past_24mths         Number of trades opened in past 24 months.
2            addr_state  The state provided by the borrower in the loan...
3              all_util              Balance to credit limit on all trades
4            annual_inc  The self-reported annual income provided by th...


In [189]:
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [190]:
# COdigo guardar

# Guardar el DataFrame en formato pickle
datos_dict.to_pickle("datos_dict.pkl")

print("DataFrame guardado como 'datos_dict.pkl'")

DataFrame guardado como 'datos_dict.pkl'


In [191]:
# Codigo para cargar
# Cargar el DataFrame desde el archivo pickle
datos_dict_cargado = pd.read_pickle("datos_dict.pkl")

print("DataFrame cargado desde 'datos_dict.pkl'")
print(datos_dict_cargado.head())

DataFrame cargado desde 'datos_dict.pkl'
                feature                                        description
0        acc_now_delinq  The number of accounts on which the borrower i...
1  acc_open_past_24mths         Number of trades opened in past 24 months.
2            addr_state  The state provided by the borrower in the loan...
3              all_util              Balance to credit limit on all trades
4            annual_inc  The self-reported annual income provided by th...


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [192]:
# Assuming your DataFrame is already loaded as 'datos_dict'
df = pd.DataFrame(loans)

unique_dtypes = df.dtypes.unique()

print("Unique Data Types in the DataFrame:")
for dtype in unique_dtypes:
    print(dtype)


Unique Data Types in the DataFrame:
int64
float64
object


In [193]:
# Manejos de tipos 1
# Tu codigo aqui
# Get a list of columns with data type int64
int64_columns = [col for col in df.columns if df[col].dtype == 'int64']

if int64_columns:
    for column in int64_columns:
        print(f"\nProcessing column: {column}")

        # Convert to numeric, ignoring errors (non-numeric values -> NaN)
        df[column] = pd.to_numeric(df[column], errors='coerce')

        # Drop NaN values to retain integer data type
        df[column].dropna(inplace=True)

        # Calculate basic statistics
        max_value = df[column].max()
        min_value = df[column].min()
        avg_value = df[column].mean()

        # Calculate quantiles (25th, 50th (median), and 75th percentiles)
        quantiles = df[column].quantile([0.25, 0.5, 0.75])

        # Display results
        print(f"Max: {max_value}")
        print(f"Min: {min_value}")
        print(f"Average: {avg_value}")
        print("\nQuantiles:")
        print(quantiles)
else:
    print("No columns with data type 'int64' found in the DataFrame.")




Processing column: Unnamed: 0
Max: 99999
Min: 0
Average: 49999.5

Quantiles:
0.25    24999.75
0.50    49999.50
0.75    74999.25
Name: Unnamed: 0, dtype: float64

Processing column: id
Max: 38098114
Min: 57167
Average: 30299954.27891

Quantiles:
0.25    27370147.0
0.50    30525558.5
0.75    34382007.0
Name: id, dtype: float64


In [194]:
# Manejos de tipos 2
# Tu codigo aqui

def cast_column_to_int32(df, column_name):
    """
    Casts the specified column in the DataFrame to in2t32.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the column to cast.
    column_name (str): The name of the column to cast.

    Returns:
    pd.DataFrame: A new DataFrame with the specified column cast to int23.
    """
    try:
        # Cast the specified column to int32
        df[column_name] = df[column_name].astype('int32')
        print(f"Column '{column_name}' successfully cast to int32.")
    except KeyError:
        print(f"Error: Column '{column_name}' not found in the DataFrame.")
    except ValueError as e:
        print(f"Error casting column '{column_name}': {e}")

    return df

cast_column_to_int32(df, 'id')
cast_column_to_int32(df, 'Unnamed: 0')


Column 'id' successfully cast to int32.
Column 'Unnamed: 0' successfully cast to int32.


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425.0,21425.0,21425.0,60 months,15.59,516.36,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800.0,12800.0,12800.0,60 months,17.14,319.08,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650.0,7650.0,7650.0,36 months,13.66,260.20,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400.0,8400.0,8400.0,36 months,9.17,267.79,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000.0,10000.0,10000.0,36 months,12.99,336.90,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000.0,30000.0,30000.0,60 months,20.99,811.44,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475.0,8475.0,8475.0,36 months,24.99,336.92,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
# Get a list of columns with data type float64
int64_columns = [col for col in df.columns if df[col].dtype == 'float64']

if int64_columns:
    for column in int64_columns:
        print(f"\nProcessing column: {column}")

        # Convert to numeric, ignoring errors (non-numeric values -> NaN)
        df[column] = pd.to_numeric(df[column], errors='coerce')

        # Drop NaN values to retain integer data type
        df[column].dropna(inplace=True)

        # Calculate basic statistics
        max_value = df[column].max()
        min_value = df[column].min()
        avg_value = df[column].mean()

        # Calculate quantiles (25th, 50th (median), and 75th percentiles)
        quantiles = df[column].quantile([0.25, 0.5, 0.75])

        # Display results
        print(f"Max: {max_value}")
        print(f"Min: {min_value}")
        print(f"Average: {avg_value}")
        print("\nQuantiles:")
        print(quantiles)
else:
    print("No columns with data type 'int64' found in the DataFrame.")


Processing column: member_id
Max: nan
Min: nan
Average: nan

Quantiles:
0.25   NaN
0.50   NaN
0.75   NaN
Name: member_id, dtype: float64

Processing column: loan_amnt
Max: 35000.0
Min: 1000.0
Average: 14886.93

Quantiles:
0.25     8000.0
0.50    13050.0
0.75    20000.0
Name: loan_amnt, dtype: float64

Processing column: funded_amnt
Max: 35000.0
Min: 1000.0
Average: 14886.93

Quantiles:
0.25     8000.0
0.50    13050.0
0.75    20000.0
Name: funded_amnt, dtype: float64

Processing column: funded_amnt_inv
Max: 35000.0
Min: 1000.0
Average: 14883.9105

Quantiles:
0.25     8000.0
0.50    13050.0
0.75    20000.0
Name: funded_amnt_inv, dtype: float64

Processing column: int_rate
Max: 26.06
Min: 6.0
Average: 13.2780734

Quantiles:
0.25    10.15
0.50    12.99
0.75    15.61
Name: int_rate, dtype: float64

Processing column: installment
Max: 1408.13
Min: 30.42
Average: 437.3318244000001

Quantiles:
0.25    261.64
0.50    380.18
0.75    573.32
Name: installment, dtype: float64

Processing column: a

In [196]:
col_int16= ['delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 
            'pub_rec', 'total_acc', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 
            'acc_now_delinq', 'acc_open_past_24mths', 'chargeoff_within_12_mths', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 
            'mths_since_recent_bc', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 
            'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 
            'num_tl_op_past_12m', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'deferral_term', 'hardship_length', 'hardship_dpd', 'settlement_term', '']
col_int32= ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'annual_inc', 'revol_bal', 'total_rec_prncp', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
            'avg_cur_bal', 'bc_open_to_buy', 'tot_hi_cred_lim', 'total_bc_limit', 'total_il_high_credit_limit', '']
col_float16= ['member_id', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m',
              'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 
              'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 
              'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 
              'sec_app_mths_since_last_major_derog', '']
col_float32= ['int_rate', 'installment', 'dti', 'revol_util', 'out_prncp', 'out_prncp_inv', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 
              'bc_util', 'pct_tl_nvr_dlq', 'hardship_amount', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 
              'settlement_amount', 'settlement_percentage', ]


def cast_columns(df, col_int16=None, col_int32=None, col_float16=None, col_float32=None):
    """
    Casts columns in the DataFrame to specified data types.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the columns to cast.
    col_int16 (list): List of columns to cast to int16.
    col_int32 (list): List of columns to cast to int32.
    col_float16 (list): List of columns to cast to float16.
    col_float32 (list): List of columns to cast to float32.

    Returns:
    pd.DataFrame: A new DataFrame with the specified columns cast to the given data types.
    """
    # Helper function to cast a list of columns to a given dtype
    def cast_to_type(columns, dtype):
        for col in columns:
            try:
                df[col] = df[col].astype(dtype)
                print(f"Column '{col}' successfully cast to {dtype}.")
            except KeyError:
                print(f"Error: Column '{col}' not found in the DataFrame.")
            except ValueError as e:
                print(f"Error casting column '{col}': {e}")

    # Cast each group of columns to the specified data type
    if col_int16:
        cast_to_type(col_int16, 'int16')
    if col_int32:
        cast_to_type(col_int32, 'int32')
    if col_float16:
        cast_to_type(col_float16, 'float16')
    if col_float32:
        cast_to_type(col_float32, 'float32')

    return df

cast_columns(df, col_int16, col_int32, col_float16, col_float32)


Column 'delinq_2yrs' successfully cast to int16.
Column 'fico_range_low' successfully cast to int16.
Column 'fico_range_high' successfully cast to int16.
Column 'inq_last_6mths' successfully cast to int16.
Error casting column 'mths_since_last_delinq': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_last_record': Cannot convert non-finite values (NA or inf) to integer
Column 'open_acc' successfully cast to int16.
Column 'pub_rec' successfully cast to int16.
Column 'total_acc' successfully cast to int16.
Column 'last_fico_range_high' successfully cast to int16.
Column 'last_fico_range_low' successfully cast to int16.
Column 'collections_12_mths_ex_med' successfully cast to int16.
Error casting column 'mths_since_last_major_derog': Cannot convert non-finite values (NA or inf) to integer
Column 'policy_code' successfully cast to int16.
Column 'acc_now_delinq' successfully cast to int16.
Column 'acc_open_past_24mths' successfully cast to int16.
Colu

/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/.venv/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/.venv/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000,15000,15000,60 months,12.390000,336.640015,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400,10400,10400,36 months,6.990000,321.079987,A,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37842129,NaN,21425,21425,21425,60 months,15.590000,516.359985,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37612354,NaN,12800,12800,12800,60 months,17.139999,319.079987,D,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37662224,NaN,7650,7650,7650,36 months,13.660000,260.200012,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,22454240,NaN,8400,8400,8400,36 months,9.170000,267.790009,B,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99996,99996,11396920,NaN,10000,10000,10000,36 months,12.990000,336.899994,C,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99997,99997,8556176,NaN,30000,30000,30000,60 months,20.990000,811.440002,E,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
99998,99998,24023408,NaN,8475,8475,8475,36 months,24.990000,336.920013,F,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


columnas en las que hubo error por presencia de Nan. Los vamos a cambiar en la imputacion. 

Error casting column 'mths_since_last_delinq': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_last_record': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_last_major_derog': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mo_sin_old_il_acct': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_recent_bc': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_recent_bc_dlq': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_recent_inq': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'mths_since_recent_revol_delinq': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'num_tl_120dpd_2m': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'percent_bc_gt_75': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'deferral_term': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'hardship_length': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'hardship_dpd': Cannot convert non-finite values (NA or inf) to integer
Error casting column 'settlement_term': Cannot convert non-finite values (NA or inf) to integer

In [197]:
boolean_columns = ['hardship_flag', 'debt_settlement_flag']

# Convert columns to boolean
for column in boolean_columns:
    if column in df.columns:
        df[column] = df[column].apply(lambda x: True if x == 'Y' else False if x == 'N' else None)
        df[column] = df[column].astype('boolean')  # Ensure it's the correct boolean type
    else:
        print(f"Column '{column}' not found in the DataFrame.")

In [198]:
# Assuming your DataFrame is loaded as 'df'

# Get a list of all columns
column_list = df.columns.tolist()

# Print the list of columns
print("List of all columns:")
print(column_list)


List of all columns:
['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_stat

In [199]:
# column_types =

# Assuming your DataFrame is already loaded as 'df'
print("Column Names and Data Types:")
for column in df.columns:
    print(f"Column: {column}, Data Type: {df[column].dtype}")

unique_dtypes = df.dtypes.unique()

print("Unique Data Types in the DataFrame:")
for dtype in unique_dtypes:
    print(dtype)

Column Names and Data Types:
Column: Unnamed: 0, Data Type: int32
Column: id, Data Type: int32
Column: member_id, Data Type: float16
Column: loan_amnt, Data Type: int32
Column: funded_amnt, Data Type: int32
Column: funded_amnt_inv, Data Type: int32
Column: term, Data Type: object
Column: int_rate, Data Type: float32
Column: installment, Data Type: float32
Column: grade, Data Type: object
Column: sub_grade, Data Type: object
Column: emp_title, Data Type: object
Column: emp_length, Data Type: object
Column: home_ownership, Data Type: object
Column: annual_inc, Data Type: int32
Column: verification_status, Data Type: object
Column: issue_d, Data Type: object
Column: loan_status, Data Type: object
Column: pymnt_plan, Data Type: object
Column: desc, Data Type: object
Column: purpose, Data Type: object
Column: title, Data Type: object
Column: zip_code, Data Type: object
Column: addr_state, Data Type: object
Column: dti, Data Type: float32
Column: delinq_2yrs, Data Type: int16
Column: earlies

Unique Data Types in the DataFrame:
int16
int32
float16
object
float32
float64
boolean

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputacion

In [200]:


# Select columns with dtype 'object'
object_columns = df.select_dtypes(include=['object'])

print("Columns with dtype 'object':")
for column in object_columns.columns:
    print(f"\nColumn: {column}")
    print(object_columns[column].head(10))  # Display first 10 rows (or modify as needed)


Columns with dtype 'object':

Column: term
0     60 months
1     36 months
2     60 months
3     60 months
4     36 months
5     36 months
6     36 months
7     60 months
8     36 months
9     36 months
Name: term, dtype: object

Column: grade
0    C
1    A
2    D
3    D
4    C
5    C
6    B
7    B
8    C
9    B
Name: grade, dtype: object

Column: sub_grade
0    C1
1    A3
2    D1
3    D4
4    C3
5    C3
6    B5
7    B4
8    C4
9    B4
Name: sub_grade, dtype: object

Column: emp_title
0                         MANAGEMENT
1     Truck Driver Delivery Personel
2    Programming Analysis Supervisor
3          Senior Sales Professional
4               Technical Specialist
5                   Admin Specialist
6             Manufacturing Engineer
7            Foreign Service Officer
8                            Teacher
9                      Store Manager
Name: emp_title, dtype: object

Column: emp_length
0    10+ years
1      8 years
2      6 years
3    10+ years
4     < 1 year
5    10+ years

In [201]:
rows_with_desc = datos_dict[datos_dict['feature'] == 'desc']
print("Rows where 'feature' is 'desc':")
print(rows_with_desc)

Rows where 'feature' is 'desc':
   feature                                description
15    desc  Loan description provided by the borrower


In [202]:
# Assuming your DataFrame is loaded as 'df'

specific_columns = [
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term',
    'hardship_amount', 'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
    'hardship_loan_status', 'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage', 'settlement_term'
]

# Select object columns
object_columns = df.select_dtypes(include=['object']).columns

# Create a set of all columns to handle: object columns + specific columns
all_columns = set(object_columns).union(specific_columns)

# Iterate over all relevant columns and fill NaN values appropriately
for column in all_columns:
    if column in df.columns:  # Ensure column exists in the DataFrame
        if column in specific_columns:
            # Fill NaN with an empty string ('')
            df[column].fillna('', inplace=True)
        else:
            # Fill NaN with 'missing'
            df[column].fillna('missing', inplace=True)

print("NaN values filled successfully.")



/var/folders/5x/v_n3jdfd003f7mqj3m9j1k0h0000gn/T/ipykernel_80522/190917898.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna('missing', inplace=True)


NaN values filled successfully.


/var/folders/5x/v_n3jdfd003f7mqj3m9j1k0h0000gn/T/ipykernel_80522/190917898.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna('', inplace=True)
/var/folders/5x/v_n3jdfd003f7mqj3m9j1k0h0000gn/T/ipykernel_80522/190917898.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[column].fillna('', inplace=True)
/var/folders/5x/v_n3jdfd003f

In [203]:
# TUcodigo aqui

specific_columns = [
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term',
    'hardship_amount', 'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
    'hardship_loan_status', 'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage', 'settlement_term'
]

# Select all numeric columns (int, float)
numeric_columns = df.select_dtypes(include=['int', 'float']).columns

# Exclude specific columns from numeric columns using set operations
filtered_numeric_columns = list(set(numeric_columns) - set(specific_columns))
filtered_columns = [col for col in filtered_numeric_columns if df[col].dtype != 'float16']

# Replace NaN values with 0 in the filtered columns
df[filtered_columns] = df[filtered_columns].fillna(0)

print("NaN values in int and non-float16 columns replaced with 0.")

bool_columns = df.select_dtypes(include=['boolean']).columns

# Replace NaN values with False in boolean columns
df[bool_columns] = df[bool_columns].fillna(False)
print("NaN values in boolean columns replaced with False.")


NaN values in int and non-float16 columns replaced with 0.
NaN values in boolean columns replaced with False.


### Codigo para salvar y cargar JSONs

In [204]:
import json

def create_imputation_strategy_json(df):
    """
    Creates a JSON-like dictionary assigning imputation strategies and values 
    for each column in the DataFrame based on its data type and a predefined list.

    Parameters:
    df: DataFrame to analyze.

    Returns:
    JSON-like dictionary with imputation strategies and values for each column.
    """
    special_columns = ['hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term',
    'hardship_amount', 'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
    'hardship_loan_status', 'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage', 'settlement_term'
    ]

    imputation_dict = {}

    for column in df.columns:
        if column in special_columns:
            estrategia = 'identificador: en estas columnas si no se tiene el valor es porque no aplica. Porque la bandera estuvo en false, por lo tanto lo cambiamos por '''
            valor = ''
        elif df[column].dtype in [np.int64, np.int32, np.int16, np.int8, np.float32, np.float64]:
            estrategia = 'identificador: son número. Si no se registro un pago o balance, se puede interpretar como que valio 0. Su asuencia se puede interpretar como un 0. Son balances o cantidades'
            valor = 0
        elif df[column].dtype == np.float16:
            estrategia = 'None: estas son columnas llenas de Nan, por lo que las dejamos así. Aunque podemos esperar ciertos valores por las descripciones de las columnas, lo deje asi porque el Nan tambien indica que no hay dato y nos permite idneitficar estas columnas.'
            valor = 'NaN'
        elif df[column].dtype == 'object':
            estrategia = 'identificador: en estas columnas, la ausencia del dato es que falta. Debe de estar ahí, porque lo tiene (como la colificacion o fecha), pero no se registró. Por lo tanto usamos missing'
            valor = 'missing'
        elif df[column].dtype == 'boolean':
            estrategia = 'identificador: en estas columnas, la ausencia del dato significa que no fue verdadero, por lo tanto, lo podemos interpretar como false'
            valor = 'False'
        else:
            estrategia = 'unknown'
            valor = None

        imputation_dict[column] = {'estrategia': estrategia, 'valor': valor}

    return imputation_dict

def save_json(data, filename='imputation_strategy.json'):
    """
    Saves the dictionary as a JSON file.

    Parameters:
    data: Dictionary to save.
    filename: Name of the JSON file (default: 'imputation_strategy.json').
    """
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"JSON saved as {filename}")



result = create_imputation_strategy_json(df)
save_json(result)

# Download the file
from IPython.display import FileLink
FileLink('imputation_strategy.json')

JSON saved as imputation_strategy.json


/Users/emiliahernandez/Desktop/fuentes/fdd_o24/tareas/208115/pandas/imputation_strategy.json

EDA


Todas la columnas llenas de Nan o sus equivalentes son inservibles

In [205]:
# Create a list of columns where all values are NaN
nan_columns = df.columns[df.isna().all()].tolist()

# Print the list of columns filled with NaN
print("List of columns completely filled with NaN:")
print(nan_columns)

target_values = ['0', 'missing', '', False]

# Create a list of columns where all values match one of the target values
columns_with_target_values = [
    col for col in df.columns 
    if df[col].astype(str).isin(target_values).all()
]

# Print the list of columns
print("Columns filled entirely with '0', 'missing',Fasle or '':")
print(columns_with_target_values)


List of columns completely filled with NaN:
['member_id', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'sec_app_mths_since_last_major_derog']
Columns filled entirely with '0', 'missing',Fasle or '':
[]


In [206]:
def find_mismatched_rows(df):
    """
    Finds and prints the rows where the values in 'Unnamed: 0' 
    do not match the DataFrame's index.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.

    Returns:
    DataFrame: A DataFrame with the mismatched rows, if any.
    """
    if 'Unnamed: 0' not in df.columns:
        print("The column 'Unnamed: 0' does not exist in the DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame

    # Identify mismatched rows
    mismatched_rows = df[df['Unnamed: 0'] != df.index]

    if not mismatched_rows.empty:
        print("Mismatched rows found:")
        print(mismatched_rows)
    else:
        print("All values in 'Unnamed: 0' match the index.")

    return mismatched_rows

# Example usage
# Assuming your DataFrame is loaded as 'df'
mismatches = find_mismatched_rows(df)

# Optional: If you want to see the rows with differences only


All values in 'Unnamed: 0' match the index.


Por lo tanto, la columna Unnamed: 0 es igual al index, por lo que es inservible

Buscamos columnas que tengas los mismos valores en cada row

In [207]:
from collections import defaultdict

def find_identical_column_sets(df):
    """
    Identifies sets of columns with identical values across all rows.

    Parameters:
    df (pd.DataFrame): The DataFrame to analyze.

    Returns:
    list: A list of sets, each containing columns with identical values across all rows.
    """
    if df.empty:
        print("The DataFrame is empty.")
        return []

    # Dictionary to store groups of columns with identical values (as a tuple of values)
    value_groups = defaultdict(set)

    # Iterate over all columns to group them by their values as a tuple
    for column in df.columns:
        # Convert the column's values to a tuple (to use as a key)
        # Handle NaN values by replacing them with a unique marker (e.g., 'NaN')
        column_values = tuple(df[column].apply(lambda x: 'NaN' if pd.isna(x) else x))
        
        # Group columns with identical values
        value_groups[column_values].add(column)

    # Extract only the sets with more than one column (i.e., identical groups)
    identical_sets = [columns for columns in value_groups.values() if len(columns) > 1]

    if not identical_sets:
        print("No identical column sets found.")
        return []

    return identical_sets

# Example usage
identical_column_sets = find_identical_column_sets(df)

# Print the results
print("Identical column sets:")
for i, column_set in enumerate(identical_column_sets, 1):
    print(f"Set {i}: {column_set}")


Identical column sets:
Set 1: {'open_act_il', 'sec_app_revol_util', 'sec_app_chargeoff_within_12_mths', 'member_id', 'sec_app_open_acc', 'mths_since_rcnt_il', 'max_bal_bc', 'sec_app_fico_range_low', 'dti_joint', 'all_util', 'total_cu_tl', 'sec_app_earliest_cr_line', 'open_rv_24m', 'il_util', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 'open_il_12m', 'open_rv_12m', 'inq_fi', 'sec_app_fico_range_high', 'sec_app_mths_since_last_major_derog', 'revol_bal_joint', 'inq_last_12m', 'sec_app_open_act_il', 'open_acc_6m', 'annual_inc_joint', 'sec_app_inq_last_6mths', 'total_bal_il', 'sec_app_collections_12_mths_ex_med', 'verification_status_joint', 'open_il_24m'}
Set 2: {'funded_amnt', 'loan_amnt'}
Set 3: {'deferral_term', 'hardship_length'}


In [208]:
# Definir las columnas a eliminar (pueden ser irrelevantes o innecesarias)
columns_to_drop = ['open_act_il', 'sec_app_revol_util', 'sec_app_chargeoff_within_12_mths', 'member_id', 'sec_app_open_acc', 'mths_since_rcnt_il', 'max_bal_bc', 'sec_app_fico_range_low', 'dti_joint', 'all_util', 'total_cu_tl', 'sec_app_earliest_cr_line', 'open_rv_24m', 'il_util', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 'open_il_12m', 'open_rv_12m', 'inq_fi', 'sec_app_fico_range_high', 'sec_app_mths_since_last_major_derog', 'revol_bal_joint', 'inq_last_12m', 'sec_app_open_act_il', 'open_acc_6m', 'annual_inc_joint', 'sec_app_inq_last_6mths', 'total_bal_il', 'sec_app_collections_12_mths_ex_med', 'verification_status_joint', 'open_il_24m', 
                  'funded_amnt', 'loan_amnt', 'funded_amnt', 'loan_amnt', 'Unnamed: 0']  # Ajusta según tu caso

# Eliminar las columnas
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

print(f"Columnas eliminadas: {columns_to_drop}")
print(f"DataFrame shape after dropping columns: {df_cleaned.shape}")


Columnas eliminadas: ['open_act_il', 'sec_app_revol_util', 'sec_app_chargeoff_within_12_mths', 'member_id', 'sec_app_open_acc', 'mths_since_rcnt_il', 'max_bal_bc', 'sec_app_fico_range_low', 'dti_joint', 'all_util', 'total_cu_tl', 'sec_app_earliest_cr_line', 'open_rv_24m', 'il_util', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 'open_il_12m', 'open_rv_12m', 'inq_fi', 'sec_app_fico_range_high', 'sec_app_mths_since_last_major_derog', 'revol_bal_joint', 'inq_last_12m', 'sec_app_open_act_il', 'open_acc_6m', 'annual_inc_joint', 'sec_app_inq_last_6mths', 'total_bal_il', 'sec_app_collections_12_mths_ex_med', 'verification_status_joint', 'open_il_24m', 'funded_amnt', 'loan_amnt', 'funded_amnt', 'loan_amnt', 'Unnamed: 0']
DataFrame shape after dropping columns: (100000, 117)


In [210]:
df_cleaned

,id,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,38098114,15000,60 months,12.390000,336.640015,C,C1,MANAGEMENT,10+ years,RENT,...,,,Cash,False,,,,,,
1,36805548,10400,36 months,6.990000,321.079987,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,...,,,Cash,False,,,,,,
2,37842129,21425,60 months,15.590000,516.359985,D,D1,Programming Analysis Supervisor,6 years,RENT,...,,,Cash,False,,,,,,
3,37612354,12800,60 months,17.139999,319.079987,D,D4,Senior Sales Professional,10+ years,MORTGAGE,...,,,Cash,False,,,,,,
4,37662224,7650,36 months,13.660000,260.200012,C,C3,Technical Specialist,< 1 year,RENT,...,,,Cash,False,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22454240,8400,36 months,9.170000,267.790009,B,B1,Software Engineer,2 years,MORTGAGE,...,,,Cash,False,,,,,,
99996,11396920,10000,36 months,12.990000,336.899994,C,C1,Assistant Professor of English,3 years,RENT,...,,,Cash,False,,,,,,
99997,8556176,30000,60 months,20.990000,811.440002,E,E4,Facilities Tech 3,10+ years,RENT,...,,,Cash,False,,,,,,
99998,24023408,8475,36 months,24.990000,336.920013,F,F4,Records Examiner Analyst,10+ years,RENT,...,,,Cash,False,,,,,,


Imputamos con el JSON

In [211]:
with open('imputation_strategy.json', 'r') as f:
    imputation_strategies = json.load(f)

# Iterate over all columns in the DataFrame
for column in df_cleaned.columns:
    if column in imputation_strategies:
        # Use the value from JSON for imputation
        imputation_value = imputation_strategies[column]["valor"]
        df_cleaned[column] = df_cleaned[column].fillna(imputation_value)
    else:
        print('error al imputar en col: ' + column )

print("Imputation completed.")

Imputation completed.


PIPELINE

In [213]:
import random
def data_pipeline(df, seed=42):
    # 1. Quitar columnas inservibles
    columns_to_drop = ['open_act_il', 'sec_app_revol_util', 'sec_app_chargeoff_within_12_mths', 'member_id', 'sec_app_open_acc', 'mths_since_rcnt_il', 'max_bal_bc', 'sec_app_fico_range_low', 'dti_joint', 'all_util', 'total_cu_tl', 'sec_app_earliest_cr_line', 'open_rv_24m', 'il_util', 'sec_app_mort_acc', 'sec_app_num_rev_accts', 'open_il_12m', 'open_rv_12m', 'inq_fi', 'sec_app_fico_range_high', 'sec_app_mths_since_last_major_derog', 'revol_bal_joint', 'inq_last_12m', 'sec_app_open_act_il', 'open_acc_6m', 'annual_inc_joint', 'sec_app_inq_last_6mths', 'total_bal_il', 'sec_app_collections_12_mths_ex_med', 'verification_status_joint', 'open_il_24m', 
                  'funded_amnt', 'loan_amnt', 'funded_amnt', 'loan_amnt', 'Unnamed: 0']
    df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

    # 2. Imputación de valores
    with open('imputation_strategy.json', 'r') as f:
        imputation_strategies = json.load(f)

    # Iterate over all columns in the DataFrame
    for column in df_cleaned.columns:
        if column in imputation_strategies:
            # Use the value from JSON for imputation
            imputation_value = imputation_strategies[column]["valor"]
            df_cleaned[column] = df_cleaned[column].fillna(imputation_value)
        else:
            print('error al imputar en col: ' + column )
    # 3. Fijar semilla para reproducibilidad
    np.random.seed(seed)
    random.seed(seed)

    print(f"Pipeline completed. DataFrame shape: {df_cleaned.shape}")
    return df_cleaned

# Uso del pipeline
df_prepared = data_pipeline(df)
df_prepared

Pipeline completed. DataFrame shape: (100000, 117)


,id,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,38098114,15000,60 months,12.390000,336.640015,C,C1,MANAGEMENT,10+ years,RENT,...,,,Cash,False,,,,,,
1,36805548,10400,36 months,6.990000,321.079987,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,...,,,Cash,False,,,,,,
2,37842129,21425,60 months,15.590000,516.359985,D,D1,Programming Analysis Supervisor,6 years,RENT,...,,,Cash,False,,,,,,
3,37612354,12800,60 months,17.139999,319.079987,D,D4,Senior Sales Professional,10+ years,MORTGAGE,...,,,Cash,False,,,,,,
4,37662224,7650,36 months,13.660000,260.200012,C,C3,Technical Specialist,< 1 year,RENT,...,,,Cash,False,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,22454240,8400,36 months,9.170000,267.790009,B,B1,Software Engineer,2 years,MORTGAGE,...,,,Cash,False,,,,,,
99996,11396920,10000,36 months,12.990000,336.899994,C,C1,Assistant Professor of English,3 years,RENT,...,,,Cash,False,,,,,,
99997,8556176,30000,60 months,20.990000,811.440002,E,E4,Facilities Tech 3,10+ years,RENT,...,,,Cash,False,,,,,,
99998,24023408,8475,36 months,24.990000,336.920013,F,F4,Records Examiner Analyst,10+ years,RENT,...,,,Cash,False,,,,,,
